# Recommenders and Solutions Recap and Evaluation<a class="anchor" id="top"></a>

## Outline

1. [Introduction](#intro)
1. [Create Domain Recommenders](#recommenders)
1. [Create Solutions](#solutions)
1. [Evaluate Solutions](#eval)
1. [Using Evaluation Metrics](#use)
1. [Deploy a Campaign](#deploy)
1. [Create Filters](#interact)
1. [Storing useful variables](#wrapup)

To run this notebook, you need to have run [the previous notebook: `01_Data_Layer.ipynb`](01_Data_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize. At the end of that notebook, you saved some of the variable values, which you now need to load into this notebook.

## Introduction <a class="anchor" id="intro"></a>

In the previous notebook we prepared 3 different datasets that represent sample data that would exist in a Media & Entertainment applicatiopn (User interactions, Media catalog data and subscriber/user data). In order to complete this workshoop within the time set, we have already created several resources on your behalf. 

### In this notebook we will accomplish the following:

Walk you throuw how we created Video on Demand Domain Recommenders for the following use cases:

1. [More like X](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#more-like-y-use-case): recommendations for movies that are similar to a movie that you specify. With this use case, Amazon Personalize automatically filters movies the user watched based on the userId that you specify and Watch events.

1. [Top picks for you](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case): personalized content recommendations for a user that you specify. With this use case, Amazon Personalize automatically filters videos the user watched based on the userId that you specify and Watch events.

Walk you throuw how we created custom solution and solution versions for the following use case:

3. [Personalized-Ranking](https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html): will be used to rerank a list of movies.

![Workflow](images/image2.png)


In [ ]:
%store -r

Similar to the previous notebook, start by importing the relevant packages, and set up a connection to Amazon Personalize using the SDK.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# How to train your Use Case Optimized Recommenders and Solution Versions

As mentioned at the top of this notebook, a dataset group, schemas, datasets, solutions, and campaigns have already been created for you. You can open another browser tab/window to view these resources in the Personalize AWS Console.

Bellow we will walk you though the steps we used to create there resorces, since they are already created, we will only be retrieving the automated deployment variables, however you can also run this code to train reources if you have not run the automation.

Please take into account that creating these resources in your account will incur a cost and that it will take upload and training time to to these steps trough this notebook (several hours).

## Configure an S3 bucket and an IAM  role <a class="anchor" id="bucket_role"></a>
[Back to top](#top)

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook. 

By default, the Personalize service does not have permission to acccess the data we uploaded into the S3 bucket in our account. In order to grant access to the  Personalize service to read our CSVs, we need to set a Bucket Policy and create an IAM role that the Amazon Personalize service will assume. Let's set all of that up.

Use the metadata stored on the instance underlying this Amazon SageMaker notebook, to determine the region it is operating in. If you are using a Jupyter notebook outside of Amazon SageMaker, simply define the region as a string below. The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources we have been creating so far.

First, let us get the current notebook region. I you are running your Amazon personalize experiments in a different region than your notebooks, you can specify it here.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print("region:", region)

Amazon S3 bucket names are globally unique. To create a unique bucket name, the code below will append the string `personalizepocvod` to your AWS account number. Then it creates a bucket with this name in the region discovered in the previous cell. 

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizepocvod"

#getting existing buckets in the account
response = s3.list_buckets()

if bucket_name in [x['Name'] for x in response['Buckets']]:
    print("The bucket already exists.")
else:
    if region == "us-east-1":
        bucket_responese = s3.create_bucket(Bucket=bucket_name)
    else:
        bucket_responese = s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
print('bucket_name:', bucket_name)

Set the S3 bucket policy
Amazon Personalize needs to be able to read the contents of your S3 bucket. So add a bucket policy which allows that.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

try:
    bucket_current_policy = s3.get_bucket_policy(Bucket=bucket_name)['Policy']
    
except s3.exceptions.from_code('NoSuchBucketPolicy') as e:    
    print("There is no current Bucket Policy for bucket " + bucket_name)
    
except Exception as e: 
    raise(e)

if (policy == json.loads(bucket_current_policy)):
    print ("The policy is already associated with the S3 Bucket.")
else:
    print ("Adding the policy to the bucket.")
    print(s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy)))

### Create an IAM role

Amazon Personalize needs the ability to assume roles in AWS in order to have the permissions to execute certain tasks. Let's create an IAM role and attach the required policies to it so it can access data from S3. The code below attaches very permissive policies, please use more restrictive policies for any production application.

In [ ]:
iam = boto3.client("iam")

role_name = account_id+"-PersonalizeS3-Immersion-Day"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

# Create or retrieve the role:
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );
    role_arn = create_role_response["Role"]["Arn"]
    
except iam.exceptions.EntityAlreadyExistsException as e:
    print('Warning: role already exists: {}\n'.format(e))
    role_arn = iam.get_role(
        RoleName = role_name
    )["Role"]["Arn"];

print('IAM Role: {}\n'.format(role_arn))


# Attach the policy if it is not previously attached:
policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"

if (policy_arn in [ x['PolicyArn'] for x in iam.list_attached_role_policies( RoleName = role_name)['AttachedPolicies']]):
    print ('The policy {} is already attached to this role.'.format(policy_arn))
else:
    print ("Attaching the role_policy")
    attach_response = iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
    );
    print ("30s pause to allow role to be fully consistent.")
    time.sleep(30)
    print('Done.')

### Upload data to S3

Now that your Amazon S3 bucket has been created, upload the CSV file of our user-item-interaction data. 

In [ ]:
interactions_file_path = data_dir + "/" + interactions_filename

try:
    s3.get_object(
        Bucket=bucket_name,
        Key=interactions_filename,
    )
    print("{} already exists in the bucket {}".format(interactions_file_path, bucket_name))
except s3.exceptions.NoSuchKey:
    boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
    print("File {} uploaded to bucket {}".format(interactions_filename, bucket_name))

items_file_path = data_dir + "/" + items_filename
try:
    s3.get_object(
        Bucket=bucket_name,
        Key=items_filename,
    )
    print("{} already exists in the bucket {}".format(items_file_path, bucket_name))
except s3.exceptions.NoSuchKey:
    boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_file_path)
    print("File {} uploaded to bucket {}".format(items_filename, bucket_name))

users_file_path = data_dir + "/" + users_filename
try:
    s3.get_object(
        Bucket=bucket_name,
        Key=users_filename,
    )
    print("{} already exists in the bucket {}".format(users_file_path, bucket_name))
except s3.exceptions.NoSuchKey:
    boto3.Session().resource('s3').Bucket(bucket_name).Object(users_filename).upload_file(users_file_path)
    print("File {} uploaded to bucket {}".format(users_filename, bucket_name))

## Create the Dataset Group <a class="anchor" id="group_dataset"></a>
[Back to top](#top)

The highest level of isolation and abstraction with Amazon Personalize is a *dataset group*. Information stored within one of these dataset groups has no impact on any other dataset group or models created from one - they are completely isolated. This allows you to run many experiments and is part of how we keep your models private and fully trained only on your data. 

Before importing the data prepared earlier, there needs to be a dataset group and a dataset added to it that handles the interactions.

Dataset groups can house the following types of information:

* User-item-interactions
* Event streams (real-time interactions)
* User metadata
* Item metadata

We need to create the dataset group that will contain our three datasets.

### Create the Dataset Group

The following cell will create a new dataset group with the name `personalize-poc-movielens`.

In [ ]:
_get_dataset_group_arn('workshop-personalize-poc-movielens')

In [ ]:
workshop_dataset_group_name = 'workshop-personalize-poc-movielens'

try: 
    create_dataset_group_response = personalize.create_dataset_group(
        name = workshop_dataset_group_name,
        domain='VIDEO_ON_DEMAND'
    )

    workshop_dataset_group_arn = create_dataset_group_response['datasetGroupArn']
    print(json.dumps(create_dataset_group_response, indent=2))
    print ('\nCreating the Dataset Group with dataset_group_arn = {}'.format(workshop_dataset_group_arn))

except personalize.exceptions.ResourceAlreadyExistsException as e:  
    workshop_dataset_group_arn = 'arn:aws:personalize:'+region+':'+account_id+':dataset-group/'+workshop_dataset_group_name 
    print ('\nThe the Dataset Group eith dataset_group_arn = {} already exists'.format(workshop_dataset_group_arn))
    print ('\nWe will be using the existing Dataset Group dataset_group_arn = {}'.format(workshop_dataset_group_arn))
    


#### Wait for Dataset Group to Have ACTIVE Status 

Before we can use the Dataset Group in any items below it must be active. This can take a minute or two. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the dataset group every 60 seconds, up to a maximum of 3 hours.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = workshop_dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Now that you have a dataset group, you can create a dataset for the interaction data.

## Create the Interactions Schema <a class="anchor" id="interact_schema"></a>
[Back to top](#top)

Now that we've loaded and prepared our three datasets we'll configure the Amazon Personalize service to understand our data so that it can be used to train models for generating recommendations.Amazon Personalize requires a schema for each dataset so it can map the columns in our CSVs to fields for model training. Each schema is declared in JSON using the [Apache Avro](https://avro.apache.org/) format. 

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

The interactions dataset has three required columns: `ITEM_ID`, `USER_ID`, and `TIMESTAMP`. The `TIMESTAMP` represents when the user interated with an item and must be expressed in Unix timestamp format (seconds). For this dataset we also have an `EVENT_TYPE` column. These must be defined in the same order in the schema as they appear in the dataset.

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE", # "Watch", "Click", etc.
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

interactions_schema_name = 'workshop-personalize-poc-movielens-interactions-schema'

try:
    create_schema_response = personalize.create_schema(
        name = interactions_schema_name,
        schema = json.dumps(interactions_schema),
        domain='VIDEO_ON_DEMAND'
    )
    print(json.dumps(create_schema_response, indent=2))
    workshop_interactions_schema_arn = create_schema_response['schemaArn']
    print ('\nCreating the Interactions Schema with workshop_interactions_schema_arn = {}'.format(workshop_interactions_schema_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException:
    workshop_interactions_schema_arn = 'arn:aws:personalize:'+region+':'+account_id+':schema/'+interactions_schema_name 
    print('The schema {} already exists.'.format(workshop_interactions_schema_arn))
    print ('\nWe will be using the existing Interactions Shema with workshop_interactions_schema_arn = {}'.format(workshop_interactions_schema_arn))
 

### Create the Interactions Dataset

With a schema created, you can create a dataset within the dataset group. Note that this does not load the data yet, but creates a schema of what the data looks like. 

In [ ]:
interactions_dataset_name = 'workshop-personalize-poc-movielens-interactions'

try:
    dataset_type = 'INTERACTIONS'
    create_dataset_response = personalize.create_dataset(
        name = interactions_dataset_name,
        datasetType = dataset_type,
        datasetGroupArn = workshop_dataset_group_arn,
        schemaArn = workshop_interactions_schema_arn
    )

    workshop_interactions_dataset_arn = create_dataset_response['datasetArn']
    print(json.dumps(create_dataset_response, indent=2))
    print ('\nCreating the Interactions Dataset with workshop_interactions_dataset_arn = {}'.format(workshop_interactions_dataset_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException:
    workshop_interactions_dataset_arn =  'arn:aws:personalize:'+region+':'+account_id+':dataset/'+workshop_dataset_group_name+'/INTERACTIONS'
    print('The Interactions Dataset {} already exists.'.format(workshop_interactions_dataset_arn))
    print ('\nWe will be using the existing Interactions Dataset with workshop_interactions_dataset_arn = {}'.format(workshop_interactions_dataset_arn))
        

## Create the Items (Movies) Schema<a class="anchor" id="items_schema"></a>
[Back to top](#top)

First, we define a schema to tell Amazon Personalize what type of dataset we are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Our item metadata data has the following columns: `ITEM_ID`, `TITLE`, `YEAR`, `IMDB_RATING`,`IMDB_NUMBEROFVOTES`,  `PLOT`, `US_MATURITY_RATING_STRING`, `US_MATURITY_RATING`,`GENRES`, `CREATION_TIMESTAMP`, and `PROMOTION` fields. These must be defined in the same order in the schema as they appear in the dataset.

In [ ]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TITLE",
            "type": "string"
        },
        {
            "name": "YEAR",
            "type": "int"
        },
        {
            "name": "IMDB_RATING",
            "type": "int"
        },
        {
            "name": "IMDB_NUMBEROFVOTES",
            "type": "int"
        },
        {
            "name": "PLOT",
            "type": "string",
            "textual": True
        },
        {
            "name": "US_MATURITY_RATING_STRING",
            "type": "string"
        },
        {
            "name": "US_MATURITY_RATING",
            "type": "int"
        },
        {
            "name": "GENRES",
            "type": "string",
            "categorical": True
        },
        {
            "name": "CREATION_TIMESTAMP",
            "type": "long"
        },
        {
            "name": "PROMOTION",
            "type": "string"
        }
    ],
    "version": "1.0"
}

items_schema_name = 'workshop-personalize-poc-movielens-items-schema'
try:
    create_schema_response = personalize.create_schema(
        name = items_schema_name,
        schema = json.dumps(items_schema),
        domain='VIDEO_ON_DEMAND'
    )
    workshop_items_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))

    print ('\nCreating the Items Schema with workshop_items_schema_arn = {}'.format(workshop_items_schema_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException:
    workshop_items_schema_arn = 'arn:aws:personalize:'+region+':'+account_id+':schema/'+items_schema_name 
    print('The schema {} already exists.'.format(workshop_items_schema_arn))
    print ('\nWe will be using the existing Items Schema with workshop_items_schema_arn = {}'.format(workshop_items_schema_arn))
 

### Create the Items Dataset

With a schema created, you can create a dataset within the dataset group. Note that this does not load the data yet, but creates a schema of what the data looks like. 

In [ ]:
items_dataset_name = 'workshop-personalize-poc-movielens-items'

try:
    
    dataset_type = "ITEMS"
    create_dataset_response = personalize.create_dataset(
        name = items_dataset_name,
        datasetType = dataset_type,
        datasetGroupArn = workshop_dataset_group_arn,
        schemaArn = workshop_items_schema_arn
    )

    workshop_items_dataset_arn = create_dataset_response['datasetArn']
    print(json.dumps(create_dataset_response, indent=2))

    print ('\nCreating the Items Dataset with workshop_items_dataset_arn = {}'.format(workshop_items_dataset_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException:
    workshop_items_dataset_arn =  'arn:aws:personalize:'+region+':'+account_id+':dataset/'+workshop_dataset_group_name+'/ITEMS'
    print('The Items Dataset {} already exists.'.format(workshop_items_dataset_arn))
    print ('\nWe will be using the existing Items Dataset with workshop_items_dataset_arn = {}'.format(workshop_items_dataset_arn))
        

## Create the Users Schema<a class="anchor" id="users_schema"></a>
[Back to top](#top)

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Here, you will create a schema for user data, which requires the `USER_ID`, and an additonal metadata field, in this case `GENDER`. These must be defined in the same order in the schema as they appear in the dataset.

In [ ]:
users_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "MEMBERLEVEL",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}
    
users_schema_name = 'workshop-personalize-poc-movielens-users-schema'
try:
    create_schema_response = personalize.create_schema(
        name = users_schema_name,
        schema = json.dumps(users_schema),
        domain='VIDEO_ON_DEMAND'
    )
    
    workshop_users_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))

    print ('\nCreating the Users Schema with workshop_users_schema_arn = {}'.format(workshop_users_schema_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException:
    workshop_users_schema_arn = 'arn:aws:personalize:'+region+':'+account_id+':schema/'+users_schema_name 
    print('The schema {} already exists.'.format(workshop_users_schema_arn))
    print ('\nWe will be using the existing Users Schema with workshop_users_schema_arn = {}'.format(workshop_users_schema_arn))
 

### Create the Users dataset

With a schema created, you can create a dataset within the dataset group. Note that this does not load the data yet, but creates a schema of what the data looks like. 

In [ ]:
try:
    dataset_type = "USERS"
    create_dataset_response = personalize.create_dataset(
        name = "workshop-personalize-poc-movielens-users",
        datasetType = dataset_type,
        datasetGroupArn = workshop_dataset_group_arn,
        schemaArn = workshop_users_schema_arn
    )

    workshop_users_dataset_arn = create_dataset_response['datasetArn']
    print(json.dumps(create_dataset_response, indent=2))

    print ('\nCreating the Users Dataset with workshop_users_dataset_arn = {}'.format(workshop_users_dataset_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException:
    workshop_users_dataset_arn =  'arn:aws:personalize:'+region+':'+account_id+':dataset/'+workshop_dataset_group_name+'/USERS'
    print('The Users Dataset {} already exists.'.format(workshop_users_dataset_arn))
    print ('\nWe will be using the existing Users Dataset with workshop_users_dataset_arn = {}'.format(workshop_users_dataset_arn))
        

Let's wait untill all the datasets have been created.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_response = personalize.describe_dataset(
        datasetArn = workshop_interactions_dataset_arn
    )
    status =  describe_dataset_response["dataset"]['status']
    print("Interactions Dataset: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
while time.time() < max_time:
    describe_dataset_response = personalize.describe_dataset(
        datasetArn = workshop_items_dataset_arn
    )
    status =  describe_dataset_response["dataset"]['status']
    print("Items Dataset: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
while time.time() < max_time:
    describe_dataset_response = personalize.describe_dataset(
        datasetArn = workshop_users_dataset_arn
    )
    status =  describe_dataset_response["dataset"]['status']
    print("Users Dataset: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Import the interactions data <a class="anchor" id="import_interactions"></a>
[Back to top](#top)

Earlier you created the dataset group and dataset to house your information, so now you will execute an import job that will load the interactions data from the S3 bucket into the Amazon Personalize dataset. 

In [ ]:
interactions_dataset_import_jobs = personalize.list_dataset_import_jobs(
    datasetArn=workshop_interactions_dataset_arn,
    maxResults=100
)['datasetImportJobs']

interactions_import_job_name = 'workshop-personalize-poc-interactions-import'

if interactions_import_job_name in [x['jobName'] for x in interactions_dataset_import_jobs]:
    
    workshop_interactions_dataset_import_job_arn =  'arn:aws:personalize:'+region+':'+account_id+':dataset-import-job/'+interactions_import_job_name
    print('The Interactions Import Job {} already exists.'.format(workshop_interactions_dataset_import_job_arn))
    print ('\nWe will be using the existing Interactions Import Job with workshop_interactions_dataset_import_job_arn = {}'.format(workshop_interactions_dataset_import_job_arn))
        
else:
    create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = interactions_import_job_name,
        datasetArn = workshop_interactions_dataset_arn,
        dataSource = {
            "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
        },
        roleArn = role_arn
    )
    workshop_interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
    print(json.dumps(create_dataset_import_job_response, indent=2))
    
    print ('\nImporting the Interactions Data with workshop_interactions_dataset_import_job_arn = {}'.format(workshop_interactions_dataset_import_job_arn))
    


## Import the Item Metadata <a class="anchor" id="import_items"></a>
[Back to top](#top)

Earlier you created the dataset group and dataset to house your information, now you will execute an import job that will load the item data from the S3 bucket into the Amazon Personalize dataset. 

In [ ]:
items_dataset_import_jobs = personalize.list_dataset_import_jobs(
    datasetArn=workshop_items_dataset_arn,
    maxResults=100
)['datasetImportJobs']

items_import_job_name = 'workshop-personalize-poc-items-import'

if items_import_job_name in [x['jobName'] for x in items_dataset_import_jobs]:
    
    workshop_items_dataset_import_job_arn =  'arn:aws:personalize:'+region+':'+account_id+':dataset-import-job/'+items_import_job_name
    print('The Items Import Job {} already exists.'.format(workshop_items_dataset_import_job_arn))
    print ('\nWe will be using the existing Items Import Job with workshop_items_dataset_import_job_arn = {}'.format(workshop_items_dataset_import_job_arn))
        
else:
    create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = items_import_job_name,
        datasetArn = workshop_items_dataset_arn,
        dataSource = {
            "dataLocation": "s3://{}/{}".format(bucket_name, items_filename)
        },
        roleArn = role_arn
    )

    workshop_items_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
    print(json.dumps(create_dataset_import_job_response, indent=2))
    print ('\nImporting the Items Data with workshop_items_dataset_import_job_arn = {}'.format(workshop_items_dataset_import_job_arn))
    
    

## Import the User Metadata <a class="anchor" id="import_users"></a>
[Back to top](#top)

Earlier you created the dataset group and dataset to house your information, now you will execute an import job that will load the user data from the S3 bucket into the Amazon Personalize dataset. 

In [ ]:
users_dataset_import_jobs = personalize.list_dataset_import_jobs(
    datasetArn=workshop_users_dataset_arn,
    maxResults=100
)['datasetImportJobs']

users_import_job_name = 'workshop-personalize-poc-users-import'

if users_import_job_name in [x['jobName'] for x in users_dataset_import_jobs]:
    
    workshop_users_dataset_import_job_arn =  'arn:aws:personalize:'+region+':'+account_id+':dataset-import-job/'+users_import_job_name
    print('The Users Import Job {} already exists.'.format(workshop_users_dataset_import_job_arn))
    print ('\nWe will be using the existing Users Import Job with workshop_users_dataset_import_job_arn = {}'.format(workshop_users_dataset_import_job_arn))
        
else:
    create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = "workshop-personalize-poc-users-import",
        datasetArn = workshop_users_dataset_arn,
        dataSource = {
            "dataLocation": "s3://{}/{}".format(bucket_name, users_filename)
        },
        roleArn = role_arn
    )

    workshop_users_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
    print(json.dumps(create_dataset_import_job_response, indent=2))
    
    print ('\nImporting the Users Data with workshop_users_dataset_import_job_arn = {}'.format(workshop_users_dataset_import_job_arn))
    
    

Before we can use the dataset, the import job must be active. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the import job every minute, up to a maximum of 6 hours.

Importing the data can take some time, depending on the size of the dataset. In this workshop, the data import job should take around 15 minutes. While you're waiting you can learn more about Datasets and Schemas in [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html). We need to wait for the data imports to complete.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = workshop_interactions_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Interactions DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)
    
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = workshop_items_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Items DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)
    
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = workshop_users_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Users DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

### Ready... Set... Train! :

Now that the data is imported and ready for use, we will create Video on Demand Domain Recommenders for the following use cases:

1. [More like X](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#more-like-y-use-case): recommendations for movies that are similar to a movie that you specify. With this use case, Amazon Personalize automatically filters movies the user watched based on the userId that you specify and Watch events.

1. [Top picks for you](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case): personalized content recommendations for a user that you specify. With this use case, Amazon Personalize automatically filters videos the user watched based on the userId that you specify and Watch events.

We will also create a custom solution and solution versions for the following use case:

3. [Personalized-Ranking](https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html): will be used to rerank a list of movies.

![Workflow](images/image2.png)


## Create Domain Recommenders <a class="anchor" id="recommenders"></a>
[Back to top](#top)

We'll start with pre-configured VIDEO_ON_DEMAND Recommenders that match some of our core use cases. Each domain has different use cases. When you create a recommender you create it for a specific use case, and each use case has different requirements for getting recommendations.

Let us look at the Recommenders supported for this domain:

In [ ]:
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND')
display_available_recipes = available_recipes ['recipes']
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND',nextToken=available_recipes['nextToken'])#paging to get the rest of the recipes 
display_available_recipes = display_available_recipes + available_recipes['recipes']
display(display_available_recipes)

[More use cases per domain](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html).

### Create a "More like X" Recommender

We are going to create a recommender of the type "More like X". This type of recommender offers recommendations for videos that are similar to a video a user watched. With this use case, Amazon Personalize automatically filters videos the user watched based on the userId specified in the `get_recommendations` call. 

In [ ]:
recommender_more_like_x_name ='workshop-more-like-x'

try:
    create_recommender_response = personalize.create_recommender(
      name = recommender_more_like_x_name,
      recipeArn = 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
      datasetGroupArn = workshop_dataset_group_arn
    )
    workshop_recommender_more_like_x_arn = create_recommender_response["recommenderArn"]
    print (json.dumps(create_recommender_response))
    print ('\nCreating the More Like X Recommender with workshop_recommender_more_like_x_arn = {}'.format(workshop_recommender_more_like_x_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException as e:
    workshop_recommender_more_like_x_arn =  'arn:aws:personalize:'+region+':'+account_id+':recommender/'+recommender_more_like_x_name
    print('The More Like X Recommender {} already exists.'.format(workshop_recommender_more_like_x_arn))
    print ('\nWe will be using the existing More Like X Recommender with workshop_recommender_more_like_x_arn = {}'.format(workshop_recommender_more_like_x_arn))
    
    

##  Create a "Top picks for you" Recommender

We are going to create a second recommender of the type "Top picks for you". This type of recommender offers personalized streaming content recommendations for a user that you specify. With this use case, Amazon Personalize automatically filters videos the user watched based on the userId that you specify and Watch events.


In [ ]:
recommender_top_pics_for_you_name ='workshop_top_picks_for_you'

try:
    create_recommender_response = personalize.create_recommender(
      name = recommender_top_pics_for_you_name,
      recipeArn = 'arn:aws:personalize:::recipe/aws-vod-top-picks',
      datasetGroupArn = workshop_dataset_group_arn
    )
    workshop_recommender_top_pics_arn = create_recommender_response["recommenderArn"]
    print (json.dumps(create_recommender_response))
    print ('\nCreating the Top Pics For You Recommender with workshop_recommender_top_pics_arn = {}'.format(workshop_recommender_top_pics_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException as e:
    workshop_recommender_top_pics_arn =  'arn:aws:personalize:'+region+':'+account_id+':recommender/'+recommender_top_pics_for_you_name
    print('The Top Pics For You Recommender {} already exists.'.format(workshop_recommender_top_pics_arn))
    print ('\nWe will be using the existing Top Pics For You Recommender with workshop_recommender_top_pics_arn = {}'.format(workshop_recommender_top_pics_arn))
    
    

arn:aws:personalize:us-east-1:674465894274:recommender/workshop_top_picks_for_you

## Retrieve your automated deployment variables

The code below will lookup the pre-created resources using the Personalize API.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')
from script import get_dataset_group_info, filter_arns, dataset_arns, schema_arns, event_tracker_arns, campaign_arns, recommender_arns, solution_arns
datasetGroupArn = get_dataset_group_info('personalize-poc-movielens)
for dataset in dataset_arns:
    if dataset.find("INTERACTIONS") != -1:
        interactions_dataset_arn = dataset
for dataset in dataset_arns:
    if dataset.find("ITEMS") != -1:
        items_dataset_arn = dataset
for dataset in dataset_arns:
    if dataset.find("USERS") != -1:
        users_dataset_arn = dataset
for schema in schema_arns:
    if schema.find("Interactions") != -1:
        interactions_schema_arn = schema
for schema in schema_arns:
    if schema.find("User") != -1:
        users_schema_arn = schema  
for schema in schema_arns:
    if schema.find("Item") != -1:
        items_schema_arn = schema  
for recommender in recommender_arns:
    if campaign.find("Personalization") != -1:
        personalization_campaign_arn = campaign
for recommender in recommender_arns:
    if campaign.find("sims") != -1:
        sims_campaign_arn = campaign
for campaign in campaign_arns:
    if campaign.find("Ranking") != -1:
        ranking_campaign_arn = campaign
for solution in solution_arns:
    if solution.find("ranking") != -1:
        ranking_solution_arn = solution

event_tracker_arn = event_tracker_arns[0]
                                         


In [ ]:
if workshop_training_complete:
    dataset_group_arn = workshop_dataset_group_arn
    interactions_dataset_arn = workshop_interactions_dataset_arn
    rerank_solution_version_arn = workshop_rerank_solution_version_arn
    rerank_campaign_arn = workshop_rerank_campaign_arn
    recommender_top_picks_arn = workshop_recommender_top_picks_arn
    recommender_more_like_x_arn = workshop_recommender_more_like_x_arn
    items_schema_arn = workshop_items_schema_arn
    users_schema_arn = workshop_users_schema_arn
    interactions_schema_arn = workshop_interactions_schema_arn
    users_dataset_arn = workshop_users_dataset_arn
    items_dataset_arn = workshop_items_dataset_arn

### Evaluate solution versions 

Personalize calculates these metrics based on a subset of the training data. The image below illustrates how Personalize splits the data. Given 10 users, with 10 interactions each (a circle represents an interaction), the interactions are ordered from oldest to newest based on the timestamp. Personalize uses all of the interaction data from 90% of the users (blue circles) to train the solution version, and the remaining 10% for evaluation. For each of the users in the remaining 10%, 90% of their interaction data (green circles) is used as input for the call to the trained model. The remaining 10% of their data (orange circle) is compared to the output produced by the model and used to calculate the evaluation metrics.

![personalize metrics](../../static/imgs/personalize_metrics.png)

We recommend reading [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) to understand the metrics, but we have also copied parts of the documentation below for convenience.

You need to understand the following terms regarding evaluation in Personalize:

* *Relevant recommendation* refers to a recommendation that matches a value in the testing data for the particular user.
* *Rank* refers to the position of a recommended item in the list of recommendations. Position 1 (the top of the list) is presumed to be the most relevant to the user.
* *Query* refers to the internal equivalent of a GetRecommendations call.

The metrics produced by Personalize are:
* **coverage**: The proportion of unique recommended items from all queries out of the total number of unique items in the training data (includes both the Items and Interactions datasets).
* **mean_reciprocal_rank_at_25**: The [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) of the first relevant recommendation out of the top 25 recommendations over all queries. This metric is appropriate if you're interested in the single highest ranked recommendation.
* **normalized_discounted_cumulative_gain_at_K**: Discounted gain assumes that recommendations lower on a list of recommendations are less relevant than higher recommendations. Therefore, each recommendation is discounted (given a lower weight) by a factor dependent on its position. To produce the [cumulative discounted gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) at K, each relevant discounted recommendation in the top K recommendations is summed together. The normalized discounted cumulative gain (NDCG) is the DCG divided by the ideal DCG such that NDCG is between 0 - 1. (The ideal DCG is where the top K recommendations are sorted by relevance.) Amazon Personalize uses a weighting factor of 1/log(1 + position), where the top of the list is position 1. This metric rewards relevant items that appear near the top of the list, because the top of a list usually draws more attention.
* **precision_at_K**: The number of relevant recommendations out of the top K recommendations divided by K. This metric rewards precise recommendation of the relevant items.

Let's take a look at the evaluation metrics for each of the solutions produced in this notebook. Please note that your results might differ from the results described in the text of this notebook, due to the quality of the Movielens dataset. 

### Personalized ranking metrics

Retrieve the evaluation metrics for the personalized ranking solution version.

In [ ]:
top_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))


## Using Evaluation Metrics <a class="anchor" id="use"></a>
[Back to top](#top)

It is important to use evaluation metrics carefully. There are a number of factors to keep in mind.

* If there is an existing recommendation system in place, this will have influenced the user's interaction history which you use to train your new solutions. This means the evaluation metrics are biased to favor the existing solution. If you work to push the evaluation metrics to match or exceed the existing solution, you may just be pushing the User Personalization to behave like the existing solution and might not end up with something better.


Keeping in mind these factors, the evaluation metrics produced by Personalize are generally useful for two cases:
1. Comparing the performance of solution versions trained on the same recipe, but with different values for the hyperparameters and features (impression data etc)
1. Comparing the performance of solution versions trained on different recipes. Here also keep in mind that the recipes answer different use cases and comparing them to each other might not make sense in your solution.

Properly evaluating a recommendation system is always best done through A/B testing while measuring actual business outcomes. Since recommendations generated by a system usually influence the user behavior which it is based on, it is better to run small experiments and apply A/B testing for longer periods of time. Over time, the bias from the existing model will fade.

## Storing useful variables <a class="anchor" id="vars"></a>
[Back to top](#top)

Before exiting this notebook, run the following cells to save the version ARNs for use in the next notebook.

In [ ]:
%store rerank_solution_version_arn
%store recommender_top_picks_arn
%store recommender_more_like_x_arn
%store rerank_solution_arn
%store rerank_campaign_arn

You're all set to move on to the last exploratory notebook: `03_Inference_Layer.ipynb`. Open it from the browser and you can start interacting with the Recommenders and Campaign and getting recommendations!